# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

This notebook is based on a [demo notebook](https://github.com/AI4Finance-LLC/FinRL/blob/7e546667a458449ebd455d97de1ac6a0d3e123e1/FinRL_stock_trading_NeurIPS_2018.ipynb) from the examples of the FinRL library. The original description follows.

Tutorials to use OpenAI DRL to trade multiple stocks in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog uses FinRL to reproduce the paper: Practical Deep Reinforcement Learning Approach for Stock Trading, Workshop on Challenges and Opportunities for AI in Financial Services, NeurIPS 2018.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-multiple-stock-trading-7b00763b7530
* Please report any issues to our Github: https://github.com/AI4Finance-LLC/FinRL-Library/issues
* **PyTorch Version** 



<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: [Dow 30](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average) consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [ ]:
import sys

RUNNING_IN_COLAB = 'google.colab' in sys.modules

In [ ]:
if RUNNING_IN_COLAB:
    try:
        import finrl
    except ImportError:
        !pip install git+https://github.com/AI4Finance-LLC/FinRL.git@7e54666

<a id='1.2'></a>
## 2.2. Import Packages

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from finrl.config import config

In [ ]:
if RUNNING_IN_COLAB:
    %load_ext tensorboard

<a id='1.3'></a>
## 2.3. Create Folders

In [ ]:
from pathlib import Path

config_dirs = [
    config.DATA_SAVE_DIR,
    config.TRAINED_MODEL_DIR,
    config.TENSORBOARD_LOG_DIR,
    config.RESULTS_DIR,
]

for d in config_dirs:
    print(f'Creating directory {d}')
    Path(d).mkdir(exist_ok=True, parents=True)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [ ]:
config.START_DATE = '2009-01-01'
config.START_TRADE_DATE = '2019-01-01'
config.END_DATE = '2021-07-01'

In [ ]:
ticker_list = config.DOW_30_TICKER
ticker_list

In [ ]:
from finrl.marketdata.yahoodownloader import YahooDownloader

df = YahooDownloader(
    start_date=config.START_DATE,
    end_date=config.END_DATE,
    ticker_list=ticker_list,
).fetch_data()
df

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [ ]:
config.TECHNICAL_INDICATORS_LIST

In [ ]:
from finrl.preprocessing.preprocessors import FeatureEngineer

processed = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=config.TECHNICAL_INDICATORS_LIST,
    use_turbulence=True,
    user_defined_feature=False,
).preprocess_data(df)
processed

In [ ]:
# In this example, this preprocessing does nothing (we have data for all tickers
# and all days), but in general it might be useful

combination = [
    (date, ticker)
    for date in pd.date_range(config.START_DATE, config.END_DATE).astype(str).values
    for ticker in ticker_list
]

processed_full = pd.DataFrame(combination, columns=["date", "tic"]).merge(processed, on=["date", "tic"], how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date', 'tic'])

processed_full = processed_full.fillna(0)
processed_full  # the only difference compared to `processed` is column order

<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Train/trade split

In [ ]:
from finrl.preprocessing.data import data_split

train = data_split(processed_full, config.START_DATE, config.START_TRADE_DATE)
trade = data_split(processed_full, config.START_TRADE_DATE, config.END_DATE)

In [ ]:
train

In [ ]:
trade

In [ ]:
config.TECHNICAL_INDICATORS_LIST

In [ ]:
stock_dimension = len(ticker_list)
state_space = (
    1 +  # Total amount of free money we have at the moment
    stock_dimension * (  # For each ticker
        2 +  # Close price & number of items owned
        len(config.TECHNICAL_INDICATORS_LIST)
    )
)
print(f"Stock dimension (number of different stocks): {stock_dimension}\nState space (dimensionality of input): {state_space}")

In [ ]:
from finrl.env.env_stocktrading import StockTradingEnv

env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
}

env_train = StockTradingEnv(df=train, **env_kwargs)

## Environment for Training



In [ ]:
env_train_sb, initial_obs = env_train.get_sb_env()

print(env_train_sb)
print(initial_obs.shape)

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

## Model Training: 5 models, A2C, DDPG, PPO, TD3, SAC


In [ ]:
from tqdm.notebook import tqdm
from stable_baselines3.common.callbacks import BaseCallback

# Note: a more correct progress bar callback implementation can be found here:
# https://github.com/araffin/rl-tutorial-jnrr19/blob/sb3/4_callbacks_hyperparameter_tuning.ipynb

class TqdmCallback(BaseCallback):
    def __init__(self):
        super().__init__()
        self.progress_bar = None
    
    def _on_training_start(self):
        self.progress_bar = tqdm(total=self.locals['total_timesteps'])
    
    def _on_step(self):
        self.progress_bar.update(1)
        return True

    def _on_training_end(self):
        self.progress_bar.close()
        self.progress_bar = None

In [ ]:
total_timesteps_dict = {
    'a2c': 50000,
    'ddpg': 50000,
    'ppo': 50000,
    'td3': 30000,
    'sac': 80000,
}

config.PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
config.TD3_PARAMS = {
    "batch_size": 100,
    "buffer_size": 1000000, 
    "learning_rate": 0.001,
}
config.SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 1000000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

In [ ]:
if RUNNING_IN_COLAB:
    %tensorboard --logdir {config.TENSORBOARD_LOG_DIR}

### Model


In [ ]:
from finrl.model.models import DRLAgent

algo_name = 'ddpg'
model = DRLAgent(env=env_train_sb).get_model(algo_name)

# # The same can also be done without the DRLAgent class as follows:
# from stable_baselines3 import <ALGO_NAME>
# model = <ALGO_NAME>(
#     policy='MlpPolicy',
#     env=env_train_sb,
#     tensorboard_log=f"{config.TENSORBOARD_LOG_DIR}/{algo_name}",
#     verbose=1,
#     **config.<ALGO_NAME>_PARAMS,
# )

In [ ]:
model.learn(
    total_timesteps=total_timesteps_dict[algo_name],
    callback=TqdmCallback(),
    log_interval=2000,
)

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the A2C model to trade Dow Jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
insample_turbulence = train[['date', 'turbulence']].drop_duplicates(subset=['date'])
insample_turbulence

In [ ]:
insample_turbulence['turbulence'].describe()

In [ ]:
turbulence_threshold = insample_turbulence['turbulence'].max()
turbulence_threshold

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2018-12 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
env_trade = StockTradingEnv(df=trade, turbulence_threshold=turbulence_threshold, **env_kwargs)

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=model,
    environment=env_trade)

In [ ]:
df_account_value

In [ ]:
df_actions

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

## 7.1 Simple plot of portfolio value

In [ ]:
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
df_baseline = YahooDownloader(
    start_date=config.START_TRADE_DATE,
    end_date=config.END_DATE,
    ticker_list=["^DJI"]
).fetch_data()

df_baseline

In [ ]:
import datetime
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

def plot_value(df, value_col_name, label):
    plt.plot_date(
        matplotlib.dates.date2num([datetime.datetime.strptime(date, '%Y-%m-%d') for date in df['date']]),
        df[value_col_name] / df[value_col_name].iloc[0],
        fmt='-',
        label=label,
    )

plt.figure(figsize=(16, 9))
plot_value(df_account_value, 'account_value', algo_name)
plot_value(df_baseline, 'close', 'Dow Jones Industrial Average')
plt.legend()
plt.grid()

<a id='6.2'></a>
## 7.2 Statistics

In [ ]:
def get_daily_return(df, value_col_name):
    return pd.Series(
        df[value_col_name].pct_change(1).values,
        index=pd.to_datetime(df["date"]),
    ).tz_localize("UTC")

test_returns = get_daily_return(df_account_value, value_col_name="account_value")
baseline_returns = get_daily_return(df_baseline, value_col_name="close")

In [ ]:
test_returns

In [ ]:
baseline_returns

In [ ]:
import pyfolio

print("============== Baseline results ==============")
print(pyfolio.timeseries.perf_stats(baseline_returns))

print("============== Our results ===================")
print(pyfolio.timeseries.perf_stats(test_returns))

<a id='6.3'></a>
## 7.3 Complex plots

In [ ]:
pyfolio.create_full_tear_sheet(
    returns=test_returns,
    benchmark_rets=baseline_returns,
)